<a href="https://colab.research.google.com/github/jtghchau/TextClassification/blob/main/pytorchProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Loading MNIST
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

trainData = torchvision.datasets.MNIST(root='data', train=True, download=True, transform=transform)
testData = torchvision.datasets.MNIST(root='data', train=False, download=True, transform=transform)

trainLoader = DataLoader(trainData, batch_size=64, shuffle=True)
testLoader = DataLoader(testData, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 58.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.49MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.4MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.1MB/s]


In [3]:
#Building the multi-layer perceptron
class MLP(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, output_size=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#Building the convuluted neural network
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
#Training the MLP

!pip install tensorboard
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/mlp_experiment')
#!tensorboard --logdir==runs/mlp_experiment

mlpModel = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlpModel.parameters(), lr=0.001)
writer = SummaryWriter("logs/mlp")

for epoch in range(10):
    mlpModel.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in trainLoader:
        optimizer.zero_grad()
        outputs = mlpModel(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_loss = total_loss / len(trainLoader)

    mlpModel.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in testLoader:
            outputs = mlpModel(images)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    test_acc = correct / total

    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Acc/test", test_acc, epoch)
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Test Acc={test_acc:.4f}")


Epoch 1: Train Loss=0.2592, Train Acc=0.9229, Test Acc=0.9562
Epoch 2: Train Loss=0.1077, Train Acc=0.9665, Test Acc=0.9676
Epoch 3: Train Loss=0.0741, Train Acc=0.9762, Test Acc=0.9738
Epoch 4: Train Loss=0.0565, Train Acc=0.9816, Test Acc=0.9729
Epoch 5: Train Loss=0.0446, Train Acc=0.9849, Test Acc=0.9772
Epoch 6: Train Loss=0.0378, Train Acc=0.9870, Test Acc=0.9733
Epoch 7: Train Loss=0.0312, Train Acc=0.9895, Test Acc=0.9810
Epoch 8: Train Loss=0.0284, Train Acc=0.9908, Test Acc=0.9779
Epoch 9: Train Loss=0.0250, Train Acc=0.9916, Test Acc=0.9782
Epoch 10: Train Loss=0.0228, Train Acc=0.9921, Test Acc=0.9748


In [5]:
#Training the CNN

!pip install tensorboard
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/cnn_experiment')
#!tensorboard --logdir==runs/mlp_experiment

cnnModel = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnnModel.parameters(), lr=0.001)
writer = SummaryWriter("logs/cnn")

for epoch in range(10):
    cnnModel.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in trainLoader:
        optimizer.zero_grad()
        outputs = cnnModel(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_loss = total_loss / len(trainLoader)

    cnnModel.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in testLoader:
            outputs = cnnModel(images)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    test_acc = correct / total

    writer.add_scalar("CNN: Loss/train", train_loss, epoch)
    writer.add_scalar("CNN: Acc/test", test_acc, epoch)
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Test Acc={test_acc:.4f}")

Epoch 1: Train Loss=0.1292, Train Acc=0.9605, Test Acc=0.9862
Epoch 2: Train Loss=0.0418, Train Acc=0.9871, Test Acc=0.9875
Epoch 3: Train Loss=0.0286, Train Acc=0.9912, Test Acc=0.9859
Epoch 4: Train Loss=0.0212, Train Acc=0.9932, Test Acc=0.9906
Epoch 5: Train Loss=0.0168, Train Acc=0.9946, Test Acc=0.9911
Epoch 6: Train Loss=0.0138, Train Acc=0.9953, Test Acc=0.9912
Epoch 7: Train Loss=0.0112, Train Acc=0.9962, Test Acc=0.9892
Epoch 8: Train Loss=0.0092, Train Acc=0.9968, Test Acc=0.9911
Epoch 9: Train Loss=0.0078, Train Acc=0.9974, Test Acc=0.9921
Epoch 10: Train Loss=0.0064, Train Acc=0.9979, Test Acc=0.9922
